In [1]:
from xml.sax import make_parser, handler
import matplotlib.pyplot as plt
from lxml import etree as lxmletree
from django.template.defaultfilters import title
import time, re, pandas

from bs4 import BeautifulSoup
import urllib2, imdb, csv, os.path


##################################################################
# UNICODE READER        
##################################################################
class UnicodeDictReader:
    """
    A CSV reader which will iterate over lines in the CSV file "variable filename",
    which is encoded in the given encoding.
    """

    def __init__(self, f, dialect=csv.excel, encoding="latin-1", **kwds):
        self.encoding = encoding
        self.reader = csv.DictReader(f, dialect=dialect, **kwds)

    def next(self):
        row = self.reader.next()
        return {k: unicode(v, "latin-1") for k, v in row.iteritems()}

    def __iter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is not None:
            print exc_type, exc_value, traceback
            # return False # uncomment to pass exception through
        return self
    
    def __enter__(self):
        return self

##################################################################
# MOVIE OBJECTS
##################################################################
class Movie:
    name=""
    year=0
    rating=""
    movie_id=""
    directors=[]
    writers=[]
    cast=[]
    
    def __init__(self, _name, _year, _rating, _id):
        self.name=_name
        self.year=_year
        self.rating=_rating
        self.movie_id=_id

##################################################################
# HELPER FUNCTIONS
##################################################################
def sortAndCleanCSV(f_in):    
    print('---START SORTING AND REMOVING DUPLICATES---')
    start_time = time.time()
    with open (f_in,'r')as datafile:
        data = pandas.read_csv(datafile, sep=',')
    datafile.close()
    #data = pandas.read_csv(f_in, sep=',', iterator=True, chunksize=10000, engine='python') #in case of bigger files
    data = data.sort_values(['NAMELAST','NAMEFIRST'],ascending=True)
    data = data.drop_duplicates(subset='NAMELAST', keep='first')
    data.to_csv(f_in,index=False,mode='wb')
    
    print ("---FINISH---")   
    print("--- %s seconds ---\n" % (time.time() - start_time))

def changeChars(file_input):
    output = 'temp.csv'
    print('---START CHANGE CHARS---')
    start_time = time.time()

    with open(file_input,"r") as csvfile:
        reader = UnicodeDictReader( csvfile )   
        writefile = open(output,'wb') 
        writer=csv.writer(writefile, delimiter=',')
        writer.writerow(['NAMELAST'] + ['NAMEFIRST'])
    
        for row in reader:
            l_name= unicode(row['NAMELAST'])
            l_name = unicode.title(l_name).encode('latin-1')
        
            f_name = unicode(row['NAMEFIRST'])
            f_name = unicode.title(f_name).encode('latin-1')
            writer.writerow([l_name] + [f_name])
        writefile.close()
    os.remove(file_input)
    os.rename(output,file_input)
    print('---FINISH CHANGE CHARS---')
    print("--- %s seconds ---\n" % (time.time() - start_time))

def getAmount(path,rank,comp):    
    print('---START GET NUMBER---')
    start_time = time.time()
    if(rank==1):
        movlist = 'movies/movielist_top.csv'
    else:
        movlist = 'movies/movielist_bottom.csv'
    movielist=[]
    movielist = getMoviefiles(path)
    
    for movie in movielist:
        filename = unicode(movie,'latin-1').rsplit('/',1)[1]
        filename = unicode(filename).rsplit('.',1)[0]
        #print(movie)
        
        with open (movie,'r')as moviefile:
            data_1 = pandas.read_csv(moviefile, sep=',')
            result = Count_Row=data_1.shape[0]
            #print(result)
        moviefile.close()
        
        with open(movlist,'r') as csvfile:
            """
            for encoding look in https://docs.python.org/3/library/codecs.html#standard-encodings
            and check entity aliases
            """
            data_2 = pandas.read_csv(csvfile, sep=',',encoding = 'latin1') 
        csvfile.close() 
        
        writefile = open(movlist,'wb') 
        writer=csv.writer(writefile, delimiter=',')    
        writer.writerow(['TITLE'] + ['RATE']+ ['WHITEHOUSE']+ ['DBLP'])
    
        for row in data_2.itertuples():
            title=unicode(row[1])
            rate=unicode(row[2])
            white = unicode(row[3])
            dblp = unicode(row[4])
            
            if(title == filename and comp == 1):
                white = unicode(result) 
            
            if(title == filename and comp == 0):
                dblp = unicode(result)
            
            writer.writerow([unicode(title).encode('latin-1')] + [rate] + [white] + [dblp])
        writefile.close()
    print ("FINISH")   
    print("--- %s seconds ---" % (time.time() - start_time))

def getMoviefiles(path):    
    directorylist = os.listdir(path)
    csvList=[]
    for i in directorylist:
        if ".csv" in i:
            #print ("selected: %s"%i)
            csvList.append(path+'/'+i)
    return csvList

def pandas_mergeFiles(file_input_1,file_input_2,file_output):
    #print('---START MERGING---') 
    start_time = time.time()   
    data_1 = pandas.read_csv(file_input_1, sep=',')
    data_2 = pandas.read_csv(file_input_2,sep=',')
     
    result=pandas.merge(data_1,data_2,how='inner')
    result.to_csv(file_output,index=False)
    #print("---FINISH---")
    #print("--- %s seconds ---" % (time.time() - start_time))
    
def getMergeFiles(file_input,rank):
    print('---START MERGING---')
    start_time = time.time()
    csvList=[]
    comp = 0
    mydirectory = os.getcwd()
    if(rank == 1):
        mydirectory= mydirectory+'/movies/top'
    else:
        mydirectory= mydirectory+'/movies/bottom'
    print(file_input)
    if "Whitehouse_visits.csv" == file_input:
        comp = 1
    
    csvList=getMoviefiles(mydirectory)
    
    for csvf in csvList:
        mergepath = unicode(csvf,'latin-1').rsplit('/',1)[0]
        mergefile = unicode(csvf,'latin-1').rsplit('/',1)[1]
        output = mergepath+'/merge/'+mergefile
        output = output.replace("\\", "/")
        #print(output)
        csvf = csvf.replace("\\", "/")
        pandas_mergeFiles(file_input,csvf,output)
        #deleteDuplicates(output)    
    getAmount(mydirectory+'/merge',rank,comp)

    
    print("---FINISH---")
    print("--- %s seconds ---" % (time.time() - start_time))



##################################################################
# GET DATA FROM IMDB
##################################################################

# get access to imdb api
try:
    imdb_access = imdb.IMDb()
    print "Connection succesful"
except imdb.IMDbError, err:
    print err
    


topMovies=[]
bottomMovies=[]

# scrape the list of top250 movies from imdb 
# ------------------------------------------
def scrapeWebpage( _url ):
    page=urllib2.urlopen(_url)
    soup = BeautifulSoup(page.read(), "lxml")

    movies = []
    table = soup.find('table', attrs={'class':'chart'})
    table_body = table.find('tbody')

    rows = table_body.findAll('tr')
    movie_number = 1
    for row in rows:
        cols = row.findAll('td')

        # Get the title + year from title column
        title_column = row.find('td', attrs={'class':'titleColumn'})
        title_column_string = title_column.text.strip().encode('latin-1')#latin-1
        item_length = len(title_column)
        movie_name = title_column_string[5:item_length-12].replace("\n", " ").strip()
        movie_year = title_column_string[item_length-10:item_length-6].replace("\n", " ").strip()

        # Get the rating from the rating column
        rating_column = row.find('td', attrs={'class':'ratingColumn imdbRating'}).text.strip().encode('latin-1')#latin-1
        movie_rating = rating_column.strip()

        # Parse the id of the title from title column
        title_link = title_column.find_all('a', href=True)
        title_link_text = title_link[0]['href']
        movie_id = title_link_text[9:16]

        # create movie and add to list 
        movie = Movie(movie_name, movie_year, movie_rating, movie_id)
        if int(movie_year) >= 1950:
            movies.append(movie)
        movie_number = movie_number+1

        # Stop after X (=10) movies
        if (len(movies) == 100):
            break
            
    return movies;


def getExtraInformation(movie,rank):
    
    #print "\nMovie", movie.name, "(", movie.year, ") with rating", movie.rating, "and ID", movie.movie_id
    
    # get the missing information about the movie
    found_movie = imdb_access.get_movie(movie.movie_id)
    writeMovieList(movie,rank)   
    # cast, director, production manager, composer, writer, producer
    if ('director' in found_movie.keys()):
        count=0
        for director in found_movie['director']:
            directors = []
            name = unicode(director).encode('latin-1')
            try:
                f_name = name.rsplit(' ',1)[0]
                l_name = name.rsplit(' ',1)[1]
                directors.append(l_name)
                directors.append(f_name)
                directors.append('Director')
                saveToFiles(movie, directors, rank)
                count=count+1
            except:
                None
    #print ("%d directors added!"%count)
    if ('writer' in found_movie.keys()):
        count=0
        for writer in found_movie['writer']:
            writers = []
            name = unicode(writer).encode('latin-1')
            try:
                f_name = name.rsplit(' ',1)[0]
                l_name = name.rsplit(' ',1)[1]
                writers.append(l_name)
                writers.append(f_name)
                writers.append('Writer')
                saveToFiles(movie, writers, rank)
                count=count+1
            except:
                None
    #print ("%d writers added!"%count)
    
    if ('cast' in found_movie.keys()):
        count=0
        for actor in found_movie['cast']:
            cast = []
            name = unicode(actor).encode('latin-1')
            try:
                f_name = name.rsplit(' ',1)[0]
                l_name = name.rsplit(' ',1)[1]
                cast.append(l_name)
                cast.append(f_name)
                cast.append('Actor')
                saveToFiles(movie, cast, rank)
                count=count+1
            except:
                None
            
    #print ("%d actors added!"%count)
    
    return;
    
def writeMovieList(movie,rank):
    if(rank>0):
        path = os.getcwd()
        filename = path+"/movies/movielist_top.csv"
    else:
        path = os.getcwd()        
        filename = path+"/movies/movielist_bottom.csv"
        
    file_exists = os.path.exists(filename)
    
    if not (file_exists):
        writefile = open(filename,'ab') 
        writer=csv.writer(writefile, delimiter=',')       
        writer.writerow(['TITLE'] + ['RATE'] + ['WHITEHOUSE']+ ['DBLP'])
        writefile.close()
    
    with open(filename, "ab") as writefile:
        writer = csv.writer(writefile,delimiter=',')
        writer.writerow([movie.name] + [movie.rating])
    writefile.close()
    
def saveToFiles( movie,persons, pos ): # pos = position (0 = bottom, 1 = top)
    # save the first ten movies by files (<movie.name>.<year>.<rating-rating>.[directors|writer|cast]) for each person-Array
    
    # build filename
    if (pos == 1):
        path = os.getcwd()
        filename = path+"/movies/top/"
    else:
        path = os.getcwd()
        filename = path+"/movies/bottom/"
        
    filename = filename+movie.name+".csv"
    file_exists = os.path.exists(filename)
    #write header
    if not(file_exists):
        writefile = open(filename,'ab') 
        writer=csv.writer(writefile, delimiter=',')       
        writer.writerow(['NAMELAST'] + ['NAMEFIRST'] + ['JOB'])
        writefile.close()
        
    #store data
    with open(filename, "ab") as writefile:
        writer = csv.writer(writefile,delimiter=',')
        writer.writerow([persons[0]] + [persons[1]] + [persons[2]])
    writefile.close()

    return;    

def getIMDB():
    print('---START GET MOVIES---')
    start_time = time.time()
    topMovies = scrapeWebpage( "http://www.imdb.com/chart/top" )
    bottomMovies = scrapeWebpage( "http://www.imdb.com/chart/bottom" )
    
    print "Top Movies:"
    for movie in topMovies:
        getExtraInformation(movie,1)
    print('Got %d movies' %len(topMovies))
        
    print "Bottom Movies:"
    for movie in bottomMovies:
        getExtraInformation(movie,0)
    print('Got %d movies' %len(bottomMovies))
    
    print('---FINISH GET MOVIES---')
    print("--- %s seconds ---\n" % (time.time() - start_time))


##################################################################
# GET DATA FROM DBLP
##################################################################

##################################################################
# CONFIGS
"""
file_input = "dblp.xml"
file_output = "dblp_authors.csv"
"""
def getDBLP(f_in,f_out):
    
    print('---START READING AUTHORS---')
   
    writefile = open(f_out, 'wb')
    writefile.write('NAMELAST,NAMEFIRST\n')
    writefile.close()

    context = lxmletree.iterparse(f_in, resolve_entities=True, load_dtd=True, tag='author')
    start_time = time.time()
    # Set start value for counting elements
    
    list = []
    for event, elem in context:
        try:
            author = elem.text.encode('latin-1')
            author = re.sub('[\d]','',author)
            author = author.rstrip()
            
        except:
            author=''
        if not(author==''):
            try:
                namefirst = author.rsplit(' ',1)[0]
            except:
                namefirst=''
            try:
                namelast = author.rsplit(' ',1)[1]
            except:
                namelast=''
        list.append(namelast+","+namefirst)
        
        if (len(list)>50000):    
            writefile = open('test.csv', 'ab')  # a = add b = byte
            
            for i in list:   
                #print (i)
                writefile.write('%s\n' % i)
            writefile.close()
            list = []
        
        # It's safe to call clear() here because no descendants will be accessed
        elem.clear()
        # Also eliminate now-empty references from the root node to <Title> 
        for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]
              
    print("--- FINISH ---")   
    print("--- %s seconds ---\n" % (time.time() - start_time))


##################################################################
# GET DATA FROM WHITEHOUSE
##################################################################
def getWhitehouseValues(file_input,file_output):  
    print('---START SHRINKING WHITEHOUSE FILE---')
    start_time = time.time()
    data_1 = pandas.read_csv(file_input, sep=',', iterator=True, chunksize=10000, engine='python')
    for row in data_1:
        result=row[['NAMELAST','NAMEFIRST']]
        result.to_csv(file_output,index=False,mode='ab')
    print ("---FINISH---")   
    print("--- %s seconds ---\n" % (time.time() - start_time))

##################################################################
# GET PLOTS
##################################################################    
def getPlot():
    print('---PLOTTING RESULTS---')
    df1 = pandas.read_csv('movies/movielist_bottom.csv', sep=',')    
    df2 = pandas.read_csv('movies/movielist_top.csv', sep=',')
   
    #Scatterplot    
    ax = df1.plot.scatter(x='RATE', y='WHITEHOUSE', color='Red',s=20, label='Worst Movies',
                          figsize=(15, 10),title='Dependency Whitehouse and movie rating');
    df2.plot.scatter(x='RATE',y='WHITEHOUSE', color='Green',s=20, label='Best Movies',grid=True, ax=ax);
    
    ax = df1.plot.scatter(x='RATE', y='DBLP', color='Red',s=20, label='Worst Movies',
                          figsize=(15, 10),title='Dependency DBLP and movie rating');
    df2.plot.scatter(x='RATE',y='DBLP', color='Green',s=20, label='Best Movies',grid=True, ax=ax);
        
    #Average amount
    av_bottom_white = (df1['WHITEHOUSE'].sum()/df1['WHITEHOUSE'].count())
    av_top_white = (df2['WHITEHOUSE'].sum()/df2['WHITEHOUSE'].count()) 
    
    av = pandas.DataFrame({'TOP-MOVIES':[av_top_white], 'BOTTOM-MOVIES':[av_bottom_white]})
    av.plot(kind='bar', title='Compare average visits in whitehouse',grid=True,figsize=(15, 10))
    
    av_bottom_dblp = (df1['DBLP'].sum()/df1['DBLP'].count())
    av_top_dblp = (df2['DBLP'].sum()/df2['DBLP'].count()) 
    
    av = pandas.DataFrame({'TOP-MOVIES':[av_top_dblp], 'BOTTOM-MOVIES':[av_bottom_dblp]})
    av.plot(kind='bar', title='Compare average amount of authors paper in dblp',grid=True,figsize=(15, 10))
        
    df1 = pandas.read_csv('dblp_authors.csv', sep=',')    
    df2 = pandas.read_csv('Whitehouse_visits.csv', sep=',')
    df3 = pandas.read_csv('dblp_whitehouse.csv', sep=',')    
    
    white = (df1['NAMELAST'].count())
    dblp = (df2['NAMELAST'].count())
    av_wd = (df3['NAMELAST'].count())    
    av = pandas.DataFrame({'Whitehouse':[white], 'DBLP':[dblp], 'mixed':[av_wd]})
    av.plot(kind='barh', title='Compare visits in whitehouse from authors in dblp',figsize=(15, 10),grid=True, use_index=True,legend=True)
   
    #Creates the plots
    plt.show()

##################################################################
# RUN AREA        
##################################################################
if __name__ == '__main__':
    
    getWhitehouseValues('White_House_Visitor_Records_Requests.csv','Whitehouse_visits.csv')
    sortAndCleanCSV('Whitehouse_visits.csv')
    changeChars('Whitehouse_visits.csv')
    sortAndCleanCSV('Whitehouse_visits.csv')
    getDBLP("dblp.xml","dblp_authors.csv")
    sortAndCleanCSV('dblp_authors.csv')
    getIMDB()
    start_time = time.time()
    print('---START OVERALL MERGING---')
    getMergeFiles('Whitehouse_visits.csv',1)
    getMergeFiles('Whitehouse_visits.csv',0)
    getMergeFiles('dblp_authors.csv',1)
    getMergeFiles('dblp_authors.csv',0)
    pandas_mergeFiles('dblp_authors.csv','Whitehouse_visits.csv','dblp_whitehouse.csv')
    print("---FINISHED OVERALL MERGING---")
    print("--- %s seconds ---\n" % (time.time() - start_time))
    sortAndCleanCSV("dblp_authors.csv")
    getPlot()

Connection succesful
---START SHRINKING WHITEHOUSE FILE---
---FINISH---
--- 10.7619998455 seconds ---

---START SORTING AND REMOVING DUPLICATES---
---FINISH---
--- 6.01699995995 seconds ---

---START CHANGE CHARS---
---FINISH CHANGE CHARS---
--- 6.90899991989 seconds ---

---START SORTING AND REMOVING DUPLICATES---
---FINISH---
--- 1.95800018311 seconds ---

---START READING AUTHORS---
--- FINISH ---
--- 1530.72900009 seconds ---

---START SORTING AND REMOVING DUPLICATES---
---FINISH---
--- 0.0130000114441 seconds ---

---START GET MOVIES---
Top Movies:
Got 100 movies
Bottom Movies:
Got 99 movies
---FINISH GET MOVIES---
--- 521.320999861 seconds ---

---START OVERALL MERGING---
---START MERGING---
Whitehouse_visits.csv
---START GET NUMBER---
FINISH
--- 4.85399985313 seconds ---
---FINISH---
--- 47.9639999866 seconds ---
---START MERGING---
Whitehouse_visits.csv
---START GET NUMBER---
FINISH
--- 4.17200016975 seconds ---
---FINISH---
--- 43.8429999352 seconds ---
---START MERGING---
dbl